<a href="https://colab.research.google.com/github/mugsiemx/Home_Sales/blob/main/Home_Sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Due to Spark/Hadoop versioning issues on my machine, Run next cell's configuration
# import os
# # Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# # For example:
# # spark_version = 'spark-3.3.1'
# spark_version = 'spark-3.3.1'
# os.environ['SPARK_VERSION']=spark_version

# # Install Spark and Java
# !apt-get update
# !apt-get install openjdk-11-jdk-headless -qq > /dev/null
# !wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
# !tar xf $SPARK_VERSION-bin-hadoop3.tgz
# !pip install -q findspark

# # Set Environment Variables
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
# os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# # Start a SparkSession
# import findspark
# findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:4 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:7 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:10 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Reading package lists... Done


In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Get:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3,202 kB]
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:11 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1,346 kB]
Get:12 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1,051 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu fo

In [27]:
# Import packages
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType
import time
import datetime
from pyspark.sql.functions import year, to_date, regexp_replace

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [28]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)

# Show DataFrame
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [29]:
# revise original column data formats:
# id remains unrevised
df = df.withColumn("year_sold", year(df['date']))
# df = df.withColumn("date", to_date(regexp_replace("date", "\d{4}-\d{2}-\d{2}","yyyy-MM-dd")))
df = df.withColumn("date_built", year(df['date_built']))
df = df.withColumn("price", df["price"].cast(IntegerType()))
df = df.withColumn("bedrooms", df["bedrooms"].cast(IntegerType()))
df = df.withColumn("bathrooms", df["bathrooms"].cast(IntegerType()))
df = df.withColumn("sqft_living", df["sqft_living"].cast(IntegerType()))
df = df.withColumn("sqft_lot", df["sqft_lot"].cast(IntegerType()))
df = df.withColumn("floors", df["floors"].cast(IntegerType()))
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+---------+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|year_sold|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+---------+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|     2022|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|     2021|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|     2019|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|     2019|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|     2022|


In [30]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- date_built: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- waterfront: string (nullable = true)
 |-- view: string (nullable = true)
 |-- year_sold: integer (nullable = true)



In [31]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
# spark.sql("""
# select year, cast(AVG(price) as numeric (9, 2)) as avgPrice
# from home_sales
# where bedrooms=4
# group by year
# """
# ).show()

In [42]:
spark.sql("""
select year_sold, cast(AVG(price) as numeric (9,2)) as avgPrice
from home_sales
where bedrooms==4
group by year_sold
"""
).show()

+---------+---------+
|year_sold| avgPrice|
+---------+---------+
|     2022|296363.88|
|     2019|300263.70|
|     2020|298353.78|
|     2021|301819.44|
+---------+---------+



In [45]:
# 4. What is the average price of a home for each date the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql("""
select date_built, cast(AVG(price) as numeric (9,2)) as avgPrice
from home_sales
where bedrooms==3 and bathrooms==3
group by date_built
"""
).show()


+----------+---------+
|date_built| avgPrice|
+----------+---------+
|      2015|288770.30|
|      2013|295962.27|
|      2014|290852.27|
|      2012|293683.19|
|      2016|290555.07|
|      2010|292859.62|
|      2011|291117.47|
|      2017|292676.79|
+----------+---------+



In [46]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
spark.sql("""
select date_built, cast(AVG(price) as numeric (9,2)) as avgPrice
from home_sales
where bedrooms==3 and bathrooms==3 and floors==2 and sqft_living>= 2000
group by date_built
"""
).show()


+----------+---------+
|date_built| avgPrice|
+----------+---------+
|      2015|297609.97|
|      2013|303676.79|
|      2014|298264.72|
|      2012|307539.97|
|      2016|293965.10|
|      2010|285010.22|
|      2011|276553.81|
|      2017|280317.58|
+----------+---------+



In [ ]:
# It seems that the intention is to retrieve the average view rating for homes with prices greater than or equal to $350,000.
#The query should calculate the average price and round it to two decimal places.
#Then group the results by the "view" column, filter the results to include only average prices greater than or equal to $350,000,
# and finally order the results in descending order of the "view" column.

In [66]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""
select DISTINCT view, cast(AVG(price) as numeric (9,2)) as avgPrice
from home_sales
group by view
having avgPrice >= 350000
order by view DESC
"""
).show(100)


print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view|  avgPrice|
+----+----------+
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95|1054325.60|
|  94|1033536.20|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87|1072285.20|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82|1063498.00|
|  81|1053472.79|
|  80| 991767.38|
|  79|1009565.08|
|  78|1080649.37|
|  77|1076205.56|
|  76|1058802.78|
|  75|1114042.94|
|  74| 745077.00|
|  73| 752861.18|
|  72| 780914.67|
|  71| 775651.10|
|  70| 695865.58|
|  69| 750537.94|
|  68| 716785.44|
|  67| 737970.96|
|  66| 712475.00|
|  65| 736679.93|
|  64| 767036.67|
|  63| 711614.55|
|  62| 759150.14|
|  61| 746877.59|
|  60| 754939.65|
|  59| 791453.00|
|  58| 759764.65|
|  57| 734340.50|
|  56| 718176.40|
|  55| 771153.32|
|  54| 798684.82|
|  53| 755214.80|
|  52| 733780.26|
|  51| 788128.21|
| 100|1026669.50|
+----+----------+

--- 0.39722251892089844 sec

In [ ]:
# 7. Cache the the temporary table home_sales.
# spark.sql('cache table home_sales')

In [ ]:
# 8. Check if the table is cached.
# spark.catalog.isCached('home_sales')

In [ ]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("""
select DISTINCT view, cast(AVG(price) as numeric (9,2)) as avgPrice
from home_sales
group by view
having avgPrice >= 350000
order by view DESC
"""
).show(100)


print("--- %s seconds ---" % (time.time() - start_time))


--- 5.459785461425781e-05 seconds ---


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
# df.write.parquet('parquet_descriptions',mode='overwrite')

In [ ]:
# 11. Read the parquet formatted data.
# p_df=spark.read.parquet('parquet_descriptions')

In [ ]:
# 12. Create a temporary table for the parquet data.
# p_df.createOrReplaceTempView('p_descriptions')

In [ ]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

--- 7.104873657226562e-05 seconds ---


In [ ]:
# 14. Uncache the home_sales temporary table.
# spark.sql("uncache home_sales")

In [ ]:
# 15. Check if the home_sales is no longer cached

